## With the data ready, portfolios are simulated

In [2]:
import bt
import yfinance as yf
import pandas as pd
import numpy as np

In [24]:
monthly_returns = pd.read_excel('monthly_returns.xlsx',parse_dates=True,index_col=0)
price_index = pd.read_excel('price_index.xlsx',parse_dates=True,index_col=0)

### A function is created to simulate portfolios

In [19]:
def generate_random_portfolios(number_of_portfolios=5,number_of_companies=20):

    random_portfolios = pd.DataFrame(index=monthly_returns.index)

    for portfolio in range(0,number_of_portfolios):

        s = bt.Strategy('s1', [bt.algos.RunMonthly(),
                       bt.algos.SelectRandomly(number_of_companies),
                       bt.algos.WeighEqually(),
                       bt.algos.Rebalance()])

        test = bt.Backtest(s, price_index,initial_capital=10000000)
        res = bt.run(test)

        random_portfolios[portfolio] = res.prices
    
    return random_portfolios

### Ten thousand portfolios are simulated
#### This computations is computationally expensive, and may take more than 12 hours

In [27]:
number_of_portfolios = 10000

results = generate_random_portfolios(number_of_portfolios=number_of_portfolios)

### S&P 500 index data is added for reference

In [ ]:
spy = yf.download("SPY")["Adj Close"].pct_change()

spy = spy["1996":]

spy = spy["1996":].resample("M").agg(lambda x: (x + 1).prod() - 1)

spy[0] = 0
spy[1] = 0

results["SPX"] = (1+spy).cumprod()*100

results.to_excel("Simulations.xlsx")

### Data is saved for future access

In [44]:
results = pd.read_excel("Simulations.xlsx",index_col=0)